<a href="https://colab.research.google.com/github/sonluongvu/Palm_structure/blob/main/%08Data_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Obtaining Data

- Palmitoylation site information is obtained from SwissPalm
- sites.txt is obtained and convert to sites.csv
- sites.csv is saved at https://github.com/sonluongvu/Palm_structure

Open sites.csv

In [58]:
! wget 'https://github.com/sonluongvu/Palm_structure/raw/main/sites.csv'

--2022-01-20 21:14:38--  https://github.com/sonluongvu/Palm_structure/raw/main/sites.csv
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sonluongvu/Palm_structure/main/sites.csv [following]
--2022-01-20 21:14:39--  https://raw.githubusercontent.com/sonluongvu/Palm_structure/main/sites.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3260327 (3.1M) [text/plain]
Saving to: ‘sites.csv.1’

sites.csv.1         100%[===================>]   3.11M  --.-KB/s    in 0.06s   

2022-01-20 21:14:39 (48.3 MB/s) - ‘sites.csv.1’ saved [3260327/3260327]



In [59]:
import pandas as pd
import numpy as np
sites_path = 'https://github.com/sonluongvu/Palm_structure/raw/main/sites.csv'
sites = pd.read_csv(sites_path)
sites.shape

(7452, 26)

Number of palmitoylation sites based on organisms

In [60]:
sites.organism.value_counts()

Mus musculus                                                            4328
Homo sapiens                                                            2691
Rattus norvegicus                                                        160
Saccharomyces cerevisiae (strain ATCC 204508 / S288c)                     46
Bos taurus                                                                38
Arabidopsis thaliana                                                      30
Oryctolagus cuniculus                                                     30
Torpedo californica                                                       13
Drosophila melanogaster                                                   12
Semliki forest virus                                                       8
Canis familiaris                                                           8
Hepatitis E virus genotype 3 (isolate Human/United States/US2)             8
Sindbis virus                                                              8

- Most of the provided palmitoylation sites above are not verified by experiments
- Listed of verified sites ID is obtained from SwissPalm database

Obtain list of verified palm sites for Homo sapiens

In [61]:
! wget 'https://github.com/sonluongvu/Palm_structure/raw/main/Homo_sapiens_verified_sites.csv'

--2022-01-20 21:14:39--  https://github.com/sonluongvu/Palm_structure/raw/main/Homo_sapiens_verified_sites.csv
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sonluongvu/Palm_structure/main/Homo_sapiens_verified_sites.csv [following]
--2022-01-20 21:14:40--  https://raw.githubusercontent.com/sonluongvu/Palm_structure/main/Homo_sapiens_verified_sites.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7373 (7.2K) [text/plain]
Saving to: ‘Homo_sapiens_verified_sites.csv.1’

Homo_sapiens_verifi 100%[===================>]   7.20K  --.-KB/s    in 0s      

2022-01-20 21:14:40 (41.0 MB/s) - ‘Homo_sapi

In [62]:
Homo_sapiens_id_path = '/content/Homo_sapiens_verified_sites.csv'
Homo_sapiens_id = pd.read_csv(Homo_sapiens_id_path)
Homo_sapiens_id.shape

(648, 1)

Obtain the list of id

In [63]:
Homo_sapiens_list = []
for i in range(len(Homo_sapiens_id.index)):
  x = str(Homo_sapiens_id.iloc[i][0])
  y = 'SPALMS'+ x[7:]
  Homo_sapiens_list.append(y)

len(Homo_sapiens_list)

648

Obtain info for Homo_sapiens_list from sites

In [64]:
Homo_sapiens_sites = sites[sites['id'].isin(Homo_sapiens_list)]
Homo_sapiens_sites.head(5)

,id,hit_id,uniprot_ac,uniprot_id,uniprot_description,organism,main_gene_name,gene_names,study_id,study_organism,study_cell_type,study_subcellular_fraction,pmid,authors,title,year,hit_techniques,large_scale,hit_list,acyl_type,isoform,pos,uncertain_pos,pats,apts,site_techniques
1,SPALMS7,SPALMH5944,O95477,ABCA1_HUMAN,ATP-binding cassette sub-family A member 1 (AT...,Homo sapiens,ABCA1,"CERP,ABC1,ABCA1",45,Homo sapiens,HEK293,Total cell lysate,19556522,Singaraja RR et al.,Palmitoylation of ATP-binding cassette transpo...,2009,"ABE (Acyl-Biotin Exchange),Labeling of substra...",No,None,palmitate,NaN,1110,No,NaN,NaN,"Point mutation of substrate,Labeling of substr..."
2,SPALMS8,SPALMH5951,P27824,CALX_HUMAN,Calnexin (IP90) (Major histocompatibility comp...,Homo sapiens,CANX,CANX,51,Homo sapiens,HeLa,Total cell lysate,22314232,Lakkaraju AK et al.,Palmitoylated calnexin is a key component of t...,2012,Co-immunoprecipitation of palmitoyltransferase...,No,None,palmitate,NaN,502,No,Q9H6R6,NaN,"Point mutation of substrate,Labeling of substr..."
3,SPALMS9,SPALMH5953,P51681,CCR5_HUMAN,C-C chemokine receptor type 5 (C-C CKR-5) (CC-...,Homo sapiens,CCR5,"CMKBR5,CCR5",53,Cricetulus griseus,CHO-K1,Total cell lysate,11323418,Blanpain C et al.,Palmitoylation of CCR5 is critical for recepto...,2001,"Point mutation of substrate,Labeling of substr...",No,None,palmitate,NaN,323,No,NaN,NaN,NaN
4,SPALMS11,SPALMH5960,P24530,EDNRB_HUMAN,Endothelin receptor type B (ET-B) (ET-BR) (End...,Homo sapiens,EDNRB,"ETRB,EDNRB",58,Homo sapiens,COS,Total cell lysate,9261180,Okamoto Y et al.,Palmitoylation of human endothelinB. Its criti...,1997,"Labeling of substrate with [3H]palmitic acid,P...",No,None,palmitate,NaN,402,No,NaN,NaN,NaN
5,SPALMS18,SPALMH5978,P43119,PI2R_HUMAN,Prostacyclin receptor (Prostaglandin I2 recept...,Homo sapiens,PTGIR,"PRIPR,PTGIR",70,Homo sapiens,HEK293,Total cell lysate,12488443,Miggin SM et al.,Palmitoylation of the human prostacyclin recep...,2003,"Labeling of substrate with [3H]palmitic acid,P...",No,None,palmitate,NaN,308,No,NaN,NaN,NaN


Obtain protein sequence

In [65]:
! pip install biopython

In [66]:
import requests as r
from Bio import SeqIO
from io import StringIO 

Uniprot_ID = Homo_sapiens_sites['uniprot_ac']
Seq_list = []
for cID in Uniprot_ID:
  baseUrl="http://www.uniprot.org/uniprot/"
  currentUrl=baseUrl+cID+".fasta"
  response = r.post(currentUrl)
  cData=''.join(response.text)

  Seq=StringIO(cData)
  pSeq=list(SeqIO.parse(Seq,'fasta'))
  Seq_list.append(pSeq)

In [67]:
protein_seq_list = []
for record in Seq_list:
  protein_seq_list.append(str(record[0].seq))
Homo_sapiens_sites['Protein_seq'] = protein_seq_list
Homo_sapiens_sites

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,id,hit_id,uniprot_ac,uniprot_id,uniprot_description,organism,main_gene_name,gene_names,study_id,study_organism,study_cell_type,study_subcellular_fraction,pmid,authors,title,year,hit_techniques,large_scale,hit_list,acyl_type,isoform,pos,uncertain_pos,pats,apts,site_techniques,Protein_seq
1,SPALMS7,SPALMH5944,O95477,ABCA1_HUMAN,ATP-binding cassette sub-family A member 1 (AT...,Homo sapiens,ABCA1,"CERP,ABC1,ABCA1",45,Homo sapiens,HEK293,Total cell lysate,19556522,Singaraja RR et al.,Palmitoylation of ATP-binding cassette transpo...,2009,"ABE (Acyl-Biotin Exchange),Labeling of substra...",No,None,palmitate,NaN,1110,No,NaN,NaN,"Point mutation of substrate,Labeling of substr...",MACWPQLRLLLWKNLTFRRRQTCQLLLEVAWPLFIFLILISVRLSY...
2,SPALMS8,SPALMH5951,P27824,CALX_HUMAN,Calnexin (IP90) (Major histocompatibility comp...,Homo sapiens,CANX,CANX,51,Homo sapiens,HeLa,Total cell lysate,22314232,Lakkaraju AK et al.,Palmitoylated calnexin is a key component of t...,2012,Co-immunoprecipitation of palmitoyltransferase...,No,None,palmitate,NaN,502,No,Q9H6R6,NaN,"Point mutation of substrate,Labeling of substr...",MEGKWLLCMLLVLGTAIVEAHDGHDDDVIDIEDDLDDVIEEVEDSK...
3,SPALMS9,SPALMH5953,P51681,CCR5_HUMAN,C-C chemokine receptor type 5 (C-C CKR-5) (CC-...,Homo sapiens,CCR5,"CMKBR5,CCR5",53,Cricetulus griseus,CHO-K1,Total cell lysate,11323418,Blanpain C et al.,Palmitoylation of CCR5 is critical for recepto...,2001,"Point mutation of substrate,Labeling of substr...",No,None,palmitate,NaN,323,No,NaN,NaN,NaN,MDYQVSSPIYDINYYTSEPCQKINVKQIAARLLPPLYSLVFIFGFV...
4,SPALMS11,SPALMH5960,P24530,EDNRB_HUMAN,Endothelin receptor type B (ET-B) (ET-BR) (End...,Homo sapiens,EDNRB,"ETRB,EDNRB",58,Homo sapiens,COS,Total cell lysate,9261180,Okamoto Y et al.,Palmitoylation of human endothelinB. Its criti...,1997,"Labeling of substrate with [3H]palmitic acid,P...",No,None,palmitate,NaN,402,No,NaN,NaN,NaN,MQPPPSLCGRALVALVLACGLSRIWGEERGFPPDRATPLLQTAEIM...
5,SPALMS18,SPALMH5978,P43119,PI2R_HUMAN,Prostacyclin receptor (Prostaglandin I2 recept...,Homo sapiens,PTGIR,"PRIPR,PTGIR",70,Homo sapiens,HEK293,Total cell lysate,12488443,Miggin SM et al.,Palmitoylation of the human prostacyclin recep...,2003,"Labeling of substrate with [3H]palmitic acid,P...",No,None,palmitate,NaN,308,No,NaN,NaN,NaN,MADSCRNLTYVRGSVGPATSTLMFVAGVVGNGLALGILSARRPARP...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7447,SPALMS5507,SPALMH26226,P06734,FCER2_HUMAN,Low affinity immunoglobulin epsilon Fc recepto...,Homo sapiens,FCER2,"IGEBF,FCE2,CLEC4J,CD23A,FCER2",925,Homo sapiens,HEK293T,Total membrane,22615937,Ivaldi C et al.,Proteomic analysis of S-acylated proteins in h...,2012,Labeling of substrate with 17-ODYA (17-octadec...,No,None,palmitate,NaN,18,Yes,NaN,NaN,Labeling of substrate with 17-ODYA (17-octadec...,MEEGQYSEIEELPRRRCCRRGTQIVLLGLVTAALWAGLLTLLLLWH...
7448,SPALMS5508,SPALMH26227,P11836,CD20_HUMAN,B-lymphocyte antigen CD20 (B-lymphocyte surfac...,Homo sapiens,MS4A1,"CD20,MS4A1",925,Homo sapiens,HEK293T,Total membrane,22615937,Ivaldi C et al.,Proteomic analysis of S-acylated proteins in h...,2012,Labeling of substrate with 17-ODYA (17-octadec...,No,None,palmitate,NaN,111,Yes,NaN,NaN,Labeling of substrate with 17-ODYA (17-octadec...,MTTPRNSVNGTFPAEPMKGPIAMQSGPKPLFRRMSSLVGPTQSFFM...
7449,SPALMS5509,SPALMH26227,P11836,CD20_HUMAN,B-lymphocyte antigen CD20 (B-lymphocyte surfac...,Homo sapiens,MS4A1,"CD20,MS4A1",925,Homo sapiens,HEK293T,Total membrane,22615937,Ivaldi C et al.,Proteomic analysis of S-acylated proteins in h...,2012,Labeling of substrate with 17-ODYA (17-octadec...,No,None,palmitate,NaN,220,Yes,NaN,NaN,Labeling of substrate with 17-ODYA (17-octadec...,MTTPRNSVNGTFPAEPMKGPIAMQSGPKPLFRRMSSLVGPTQSFFM...
7450,SPALMS5512,SPALMH26235,P01112,RASH_HUMAN,GTPase HRas (H-Ras-1) (Ha-Ras) (Transforming p...,Homo sapiens,HRAS,"HRAS1,HRAS",930,Homo sapiens,HEK293T,Total cell lysate,27044110,Percher A et al.,Mass-tag labeling reveals site-specific 

In [80]:
columns = ['id', 'uniprot_ac', 'pos', 'protein_seq']
Homo_sapiens_info = Homo_sapiens_sites.filter(columns, axis = 1)
Homo_sapiens_info['protein_seq'] = protein_seq_list
Homo_sapiens_info.shape

(643, 4)

In [81]:
Homo_sapiens_info.to_csv('Homo_sapiens_info.csv', index = False)

Obtain 100 aa upstream and 100 aa downstream